In [33]:
import yaml
import os

with open('n_warehouse_21_3_real_lost_demand_separate_example.yml', 'r') as file:
    config = yaml.safe_load(file)

In [31]:
n_stores = 21
n_warehouses = 3
# Create separate configs for each store-warehouse combination
for store_idx in range(n_stores):
    for warehouse_idx in range(n_warehouses):
        config_copy = yaml.safe_load(yaml.dump(config))
        config_copy['store_params']['demand']['file_location'] = f'/user/ml4723/Prj/NIC/data_files/favorita_21_stores/separate/weekly_sales_{store_idx}.pt'
        config_copy['store_params']['underage_cost']['file_location'] = f'/user/ml4723/Prj/NIC/data_files/favorita_21_stores/separate/underage_costs_{store_idx}.pt'
        config_copy['store_params']['holding_cost']['file_location'] = f'/user/ml4723/Prj/NIC/data_files/favorita_21_stores/separate/holding_costs_{store_idx}.pt'

        # Update warehouse specific parameters
        config_copy['warehouse_params']['holding_cost']['value'] = config['warehouse_params']['holding_cost']['value'][warehouse_idx]
        config_copy['warehouse_params']['edge_initial_cost']['value'] = config['warehouse_params']['edge_initial_cost']['value'][warehouse_idx]
        
        if config['warehouse_params']['edges']['value'][warehouse_idx][store_idx] == 0:
            continue
        config_copy['warehouse_params']['edge_lead_times']['value'] = [[config['warehouse_params']['edge_lead_times']['value'][warehouse_idx][store_idx]]]
        config_copy['warehouse_params']['edges']['value'] = [[config['warehouse_params']['edges']['value'][warehouse_idx][store_idx]]]
        
        # Save config
        output_dir = '/user/ml4723/Prj/NIC/config_files/settings/separate'
        os.makedirs(output_dir, exist_ok=True)
        with open(f'{output_dir}/{store_idx}_{warehouse_idx}.yml', 'w') as file:
            yaml.dump(config_copy, file)


In [4]:
with open('n_warehouse_21_3_real_lost_demand.yml', 'w') as file:
    yaml.dump(config, file)


In [12]:
import torch
underage_costs = torch.load('/user/ml4723/Prj/NIC/data_files/favorita_21_stores/underage_costs.pt')
holding_costs = torch.load('/user/ml4723/Prj/NIC/data_files/favorita_21_stores/holding_costs.pt')

In [14]:
import torch
import os

# Load the full tensors
underage_costs = torch.load('/user/ml4723/Prj/NIC/data_files/favorita_21_stores/underage_costs.pt')
holding_costs = torch.load('/user/ml4723/Prj/NIC/data_files/favorita_21_stores/holding_costs.pt')

# Create output directory if it doesn't exist
output_dir = '/user/ml4723/Prj/NIC/data_files/favorita_21_stores/separate'
os.makedirs(output_dir, exist_ok=True)

# Split and save individual store data
for store_idx in range(21):
    # Split underage costs
    store_underage = underage_costs[:, store_idx:store_idx+1]
    torch.save(store_underage, os.path.join(output_dir, f'underage_costs_{store_idx}.pt'))
    
    # Split holding costs 
    store_holding = holding_costs[:, store_idx:store_idx+1]
    torch.save(store_holding, os.path.join(output_dir, f'holding_costs_{store_idx}.pt'))

In [3]:
import torch
import os

# Load the full tensor
data = torch.load('/user/ml4723/Prj/NIC/data_files/favorita_21_stores/weekly_sales.pt')

# Create output directory if it doesn't exist
output_dir = '/user/ml4723/Prj/NIC/data_files/favorita_21_stores/separate'
os.makedirs(output_dir, exist_ok=True)

# Split and save individual store data
for store_idx in range(21):
    store_data = data[:, store_idx:store_idx+1, :]  # Shape: (288, 1, ?)
    output_path = os.path.join(output_dir, f'weekly_sales_{store_idx}.pt')
    torch.save(store_data, output_path)

In [1]:
%load_ext autoreload
%autoreload 2

import analysis.ray_results_interpreter as rri
import glob
yml_files = [path.split('settings/')[1].replace('.yml', '') for path in glob.glob('/user/ml4723/Prj/NIC/config_files/settings/separate/*.yml')]

results_interpretor = rri.RayResultsinterpreter()
def custom_data_filler(out_row, reference_row):
    out_row['path'] = reference_row['path']

def default_condition_setter(condition_name):
    return None

sort_by = 'train_loss'
pick_row_from_run_by = 'train_loss'
conditions = {
    'config': yml_files,
}

df = results_interpretor.make_table({1: "/user/ml4723/Prj/NIC/ray_results/separate_store/data_driven_net_n_warehouses_real"},
    conditions,
    default_condition_setter,
    custom_data_filler,
    sort_by=sort_by,
    pick_row_from_run_by=pick_row_from_run_by,
    test_loss_limit=100)


# Extract store and warehouse indices from config column
df['store_index'] = df['config'].apply(lambda x: int(x.split('_')[0].split('/')[-1]))
df['warehouse_index'] = df['config'].apply(lambda x: int(x.split('_')[1]))

# Group by store_index and find warehouse with lowest dev loss for each store
best_warehouses = df.groupby('store_index').apply(lambda x: x.loc[x['Dev Loss'].idxmin()][['store_index', 'warehouse_index', 'Dev Loss']])
print("\nBest warehouse assignments by dev loss:")
print(best_warehouses)
# Get best warehouse assignments from best_warehouses DataFrame
best_assignments = {}
for _, row in best_warehouses.iterrows():
    store = row['store_index'] 
    warehouse = row['warehouse_index']
    best_assignments[store] = warehouse

# Create new edge matrix with only best warehouse connections
n_warehouses = 3
n_stores = 21
new_edges = [[0]*n_stores for _ in range(n_warehouses)]
edge_lead_times = [[3, 3, 3, 4, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2],
                   [2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 4, 3, 4, 2, 2, 2, 2, 2, 2, 3],
                   [4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 4, 4, 4, 4, 4, 3, 2]]

for store in range(n_stores):
    if store in best_assignments:
        best_warehouse = int(best_assignments[store])  # Convert float to int
        new_edges[best_warehouse][store] = 1
print("  'edge_lead_times':")
print("    'value': [")
for row in edge_lead_times:
    print("             ", row, ",")
print("            ]")
print("  'edges':")
print("    'value': [")
for row in new_edges:
    print("             ", row, ",") 
print("            ]")

/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (



Best warehouse assignments by dev loss:
             store_index  warehouse_index    Dev Loss
store_index                                          
0                      0                0 -734.874081
1                      1                1 -150.253013
2                      2                0 -386.780867
3                      3                1 -394.398438
4                      4                1 -120.980213
5                      5                0 -171.394238
6                      6                0 -269.323963
7                      7                0 -217.438113
8                      8                0 -269.214920
9                      9                0 -259.378013
10                    10                0 -217.708538
11                    11                2 -217.735498
12                    12                0 -265.235907
13                    13                0 -194.272595
14                    14                0 -795.323734
15                    15                0

/tmp/ipykernel_26030/3445438880.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_warehouses = df.groupby('store_index').apply(lambda x: x.loc[x['Dev Loss'].idxmin()][['store_index', 'warehouse_index', 'Dev Loss']])
